# Seq2Seq

이름 : 김주연

기수 : 11기

작성자 : 10 기 신재우

In [1]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [2]:
%pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 48.8 MB/s eta 0:00:00


In [2]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

ModuleNotFoundError: No module named 'konlpy'

### 경로 설정

In [6]:
path_to_folder = "/content/drive/MyDrive/[0225]RNN + Transformers_Jooyeon Kim"

In [7]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")
train_dataset = pd.read_csv(path_train, index_col = 0)
val_dataset = pd.read_csv(path_val, index_col = 0)
test_dataset = pd.read_csv(path_test, index_col = 0)

# Vocabulary

In [8]:
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [9]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [10]:
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [11]:
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10442


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [12]:
en_indices = []
for word in train_dataset['en_tokens']:
  en_indices.append(en_vocab.lookup_indices(word))

train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:
  kr_indices.append(kr_vocab.lookup_indices(word))

train_dataset['kr_indices'] = kr_indices

In [13]:
en_indices = []
for word in val_dataset['en_tokens']:
  en_indices.append(en_vocab.lookup_indices(word))

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:
  kr_indices.append(kr_vocab.lookup_indices(word))

val_dataset['kr_indices'] = kr_indices

In [14]:
en_indices = []
for word in test_dataset['en_tokens']:
  en_indices.append(en_vocab.lookup_indices(word))


test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:
  kr_indices.append(kr_vocab.lookup_indices(word))

test_dataset['kr_indices'] = kr_indices

# Data Loaders

In [15]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']
        return en_indices, kr_indices

In [16]:
batch_no = 80

#def collate_fn(batch):
    #en_indices, kr_indices = zip(*batch)
    #en_indices_padded = pad_sequence(en_indices, batch_first=True, padding_value=0)
    #kr_indices_padded = pad_sequence(kr_indices, batch_first=True, padding_value=0)

    #return en_indices_padded, kr_indices_padded
def collate_fn(batch):
    en_indices, kr_indices = zip(*batch)
    # 각 시퀀스를 텐서로 변환
    en_indices_tensor = [torch.tensor(seq, dtype=torch.long) for seq in en_indices]
    kr_indices_tensor = [torch.tensor(seq, dtype=torch.long) for seq in kr_indices]
    # pad_sequence를 사용하여 패딩 적용
    en_indices_padded = pad_sequence(en_indices_tensor, batch_first=True, padding_value=0)
    kr_indices_padded = pad_sequence(kr_indices_tensor, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [17]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_de_ids = [example["de_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [18]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 31])
Shape of kr_indices: torch.Size([80, 32])
Sample en_indices batch: tensor([[   2,   13,    8,   47,    0,    0,   21,   81,   17,  221,   14, 1093,
          111,   14,    0, 8059,  739,    4,    3,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2,    0, 5560,  185, 6509,  116,  263,   16, 2439,    5,   11,   82,
           13,    0,   36,   58,   82,    7,  199,    4,    3,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
Batch 2
Shape of en_indices: torch.Size([80, 32])
Shape of kr_indices: torch.Size([80, 34])
Sample en_indices batch: tensor([[   2,   10,   84,   30,   26,   33,    6, 4899,    5, 2140,   43,  716,
          117,   22,  422, 1093,    9, 1113,    4,    3,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2, 1205,   64,   67,   57,  116,   46, 1118,  2

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [19]:

  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
class Encoder(nn.Module):
    def __init__(self, input_size_encoder, embedding_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size_encoder, embedding_size)

        self.gru = nn.GRU(embedding_size, hidden_size, batch_first=True)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        output, hidden = self.gru(embedded)
        return output, hidden


In [20]:

  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.

  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embedding_size, hidden_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_vocab_size


        self.embedding = nn.Embedding(output_vocab_size, embedding_size)

        self.gru = nn.GRU(embedding_size, hidden_size, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, input, hidden):

        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded, hidden)
        output = self.fc(output.squeeze(1))
        return output, hidden


In [26]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = target.shape[0]
        target_len = target.shape[1]
        target_vocab_size = self.decoder.output_size

        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)

        _, hidden = self.encoder(source)

        input = target[:, 0]

        for t in range(1, target_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t, :] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = target[:, t] if teacher_force else top1
        return outputs


# Training

In [27]:
input_size_encoder = len(en_vocab)
output_size_encoder = len(kr_vocab)
embedding_size = 512
hidden_size = 1042

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_size_encoder, embedding_size, hidden_size)
decoder = Decoder(output_size_encoder, embedding_size, hidden_size)
model = Seq2Seq(encoder, decoder, device).to(device)


### 모델 확인 + Initialize 시키기

In [28]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,968,686 trainable parameters


### 하이퍼 파라미터 설정!

In [29]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [30]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 2.7753708197275797, Val Loss: 3.2580343201046897
Epoch: 2, Train Loss : 2.4741380411783855, Val Loss: 2.858134595174638
Epoch: 3, Train Loss : 2.2926608330408733, Val Loss: 2.7791347049531483
Epoch: 4, Train Loss : 2.136662625948588, Val Loss: 2.882359955045912
Epoch: 5, Train Loss : 1.9887419974009195, Val Loss: 2.8065486370571078
Epoch: 6, Train Loss : 1.8210965585708618, Val Loss: 2.8109688229031033
Epoch: 7, Train Loss : 1.6686522073745726, Val Loss: 2.8382128231109136
Epoch: 8, Train Loss : 1.52429354317983, Val Loss: 2.943733283451625
Epoch: 9, Train Loss : 1.4089463996887206, Val Loss: 3.0252160344805037
Epoch: 10, Train Loss : 1.2879384996096293, Val Loss: 3.018623772121611


파라미터 저장

In [31]:
path_to_save = os.path.join(path_to_folder, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [32]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'but', 'why', 'are', 'you', 'asking', 'this', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그런데', '이것', '을', '외', '어', '묻', '어', '보시', '는', '거', '이', '에요', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '이', '이', '한테', '에게', '이것', '있', 'ㄴ', '이', '있', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [77]:

class Encoder(nn.Module):
    def __init__(self, input_size, embedding_encoder_size, hidden_size):
      super().__init__()
      self.embedding = nn.Embedding(input_size, embedding_encoder_size)
      self.gru = nn.GRU(embedding_encoder_size, hidden_size, batch_first=True)

    def forward(self, input):
      embedded = self.embedding(input)
      output, hidden = self.gru(embedded)
      return output, hidden


In [78]:

  """
  해당 모듈에서는 Attention Value 까지 찾는 단계입니다.

  Encoder 내에서의 Output 들의 개수마다 for loop 을 진행시켜서 합산시켜서 Attention Value 를 구하는 방법도 있지만,
  Encoder Output 들을 모두 Matrix 으로 표현해서 구하는 방법도 있습니다.

  Decoder_Outputs : [Batch_size, 1, Hidden_size]
  Encoder_Outputs : [Batch_size, Sequence Length, Hidden_size]

  힌트 :
  - Decoder_Outputs 의 차원이 위와 같이 안 나온다면 차원 조정을 위해서 squeeze, unsqueeze, transpose 등의 함수들을 활용해주면 됩니다.
  - Matrix Multiplication 을 진행할 때에 Batch 까지 곱해지는 것을 방지하기 위해서 torch.bmm(x, y) 를 활용해주면 됩니다.
  - 곱해줄 때에는 그저 x * y 를 해주면 됩니다.
  - 합산할 때에는 배치도 고려해야 함으로 torch.sum(x, dim = 1) 을 활용해주면 됩니다.
  - 해당 모듈 내에서 필요한 nn. 함수는 오로지 Softmax 밖에 없습니다.
  """
class Attention(nn.Module):
    def __init__(self, hidden_size):
      super(Attention, self).__init__()
      self.hidden_size = hidden_size

      self.attn_weights = nn.Linear(hidden_size * 2, 1) # input 으로 encoder, decoder 두 개를 받기 때문

    def forward(self, decoder_outputs, encoder_outputs):

      decoder_outputs = decoder_outputs.transpose(0, 1).transpose(1, 2) #결과 decoder_outputs =  [1, hidden_size, batch_size] * encdoer_outputs = [batch_size, seq_length, hidden_size]

      attn_scores = torch.bmm(encoder_outputs, decoder_outputs) # batch 빼고 계산

      attn_weights = F.softmax(attn_scores, dim=1)

      context_vector = torch.sum(attn_weights * encoder_outputs, dim=1)

      return context_vector, attn_weights



In [79]:
class Decoder(nn.Module):
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
        super().__init__()
        self.embedding = nn.Embedding(output_size, embedding_decoder_size)
        self.gru = nn.GRU(embedding_decoder_size + hidden_size, hidden_size, batch_first=True)
        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size * 2, output_size)

        self.output_size = output_size

  def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(1)  # [Batch_size, 1]
        embedded = self.embedding(input)  # [Batch_size, 1, Embedding_decoder_size]

        context_vector, attention_weights = self.attention(hidden, encoder_outputs)
        context_vector = context_vector.unsqueeze(1)  # [Batch_size, 1, Hidden_size]

        gru_input = torch.cat((embedded, context_vector), dim=2)  # [Batch_size, 1, Embedding_decoder_size + Hidden_size]
        output, hidden = self.gru(gru_input, hidden)  # output: [Batch_size, 1, Hidden_size], hidden: [1, Batch_size, Hidden_size]

        output = output.squeeze(1) #[Batch_size, Hidden_size]

        context_vector = context_vector.squeeze(1)  # [Batch_size, Hidden_size]

        combined_output = torch.cat((output, context_vector), dim=1)
        preds = self.fc(combined_output)

        return preds, hidden, attention_weights

In [80]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
      super().__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.device = device

    def forward(self, input, target, teacher_forcing_ratio=0.5):
        batch_size = target.shape[0]
        target_seq_len = target.shape[1] #타겟 토큰 길이
        target_vocab_size = self.decoder.output_size # context vector의 차원

        # output을 저장할 tensor를 만듭니다.(처음에는 전부 0으로)
        outputs = torch.zeros(batch_size, target_seq_len, target_vocab_size).to(self.device)

        # input 문장을 encoder에 넣은 후 hidden, cell값을 구합니다.
        encoder_output, encoder_hidden = self.encoder(input)

        #decoder에 입력할 첫 번째 input
        #trg[:,0].shape = batch_size
        decoder_input = target[:, 0]  # 첫 번째 디코더 입력은 항상 <sos>

        for t in range(1, target_seq_len):
            # decoder에 입력한 첫 번째 input과 encoder_hidden을 이용하여 다음 step의 output 및 hidden state를 얻습니다.
            decoder_output, decoder_hidden, attn_weights = self.decoder(decoder_input, encoder_hidden, encoder_output)

            # prediction 저장
            outputs[:, t, :] = decoder_output

            # teacher forcing을 사용할지, 말지 결정
            teacher_force = random.random() < teacher_forcing_ratio

            # 가장 높은 확률을 갖은 값 얻기
            top1 = decoder_output.argmax(1)

            # teacher forcing의 경우에 다음 lstm에 target token 입력
            decoder_input = target[:, t] if teacher_force else top1

        return outputs


# Training

In [83]:
#input_dim = len(en_vocab)
#output_dim = len(kr_vocab)
#encoder_embedding_dim = 512
#decoder_embedding_dim = 512
#hidden_dim = 1024

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
#decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
#model = Seq2Seq(encoder, decoder, device).to(device)

input_size = len(en_vocab)
output_size = len(kr_vocab)
embedding_encoder_size = 512
embedding_decoder_size = 512
hidden_size = 1042

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_size, embedding_encoder_size, hidden_size)
decoder = Decoder(hidden_size, embedding_decoder_size, output_size)
model = Seq2Seq(encoder, decoder, device).to(device)


### 모델 확인 + Initialize 시키기

In [84]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 45,108,627 trainable parameters


### 하이퍼 파라미터 설정!

In [85]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [86]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 3.092813274383545, Val Loss: 3.6167448891533747
Epoch: 2, Train Loss : 2.694357275644938, Val Loss: 3.7363019368005177
Epoch: 3, Train Loss : 2.4832556317647296, Val Loss: 3.697466884340559
Epoch: 4, Train Loss : 2.2685981658299763, Val Loss: 3.810745844765315
Epoch: 5, Train Loss : 2.097769206047058, Val Loss: 4.008137612115769
Epoch: 6, Train Loss : 1.9571132799784343, Val Loss: 3.9245452880859375
Epoch: 7, Train Loss : 1.8558155555725098, Val Loss: 3.991516310071188
Epoch: 8, Train Loss : 1.7939866205851236, Val Loss: 3.9174297499278237
Epoch: 9, Train Loss : 1.7386742286682129, Val Loss: 4.076104088435097
Epoch: 10, Train Loss : 1.6749145720799763, Val Loss: 4.148474299718464


### 파라미터 저장

In [1]:
path_to_save = os.path.join(path_to_folder, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

NameError: name 'os' is not defined

### 적용시켜보기

In [3]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

NameError: name 'train_loader' is not defined

아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!